In [81]:
import copy
import pandas
import heapq
import math
from math import exp
from pandas import read_csv

"""Network class"""
class Node(object):
    def __init__(self,nodeid,X,Y):
        self.id = nodeid
        self.X = X
        self.Y = Y
        self.cost = -1
        return None
            
class Link(object):
    def __init__(self,linkid,st,en,FFTT):
        self.id = linkid
        self.st = st
        self.en = en
        self.FFTT = FFTT
        self.likelihood = 0
        self.weight = 0
        self.tra_vol = 0
        return None
       
class Network(object):
    def __init__(self):
        self.nodes = {}
        self.links = {}
        self.outlinks = {}
        self.inlinks = {}
        return None
    
    def addNode(self,node):
        self.nodes[node.id] = node
        return 0
    
    def addLink(self,link):
        st = link.st
        en = link.en
        self.links[link.id] = link
        if st not in self.outlinks:
            self.outlinks[st]=[]
        self.outlinks[st].append(link)
        
        if en not in self.inlinks:
            self.inlinks[en]=[]
        self.inlinks[en].append(link)
        return 0
                
"""Dijkstra"""    
def Dijkstra(net, st, en):
    nodes_Num = len(net.nodes)
    previous_node = [-1]*nodes_Num
    previous_links_id = [-1]*nodes_Num
    links_id_inSP = []
    node_cost = [1000000]*nodes_Num
    node_cost[st]=0
    net.nodes[st].cost = 0
    heap = []
    heapq.heappush(heap,(0,st))
    while(len(heap) !=0):
        current_node_cost, current_node_id = heapq.heappop(heap)
        if (net.nodes[current_node_id].cost == -1):
            net.nodes[current_node_id].cost = current_node_cost
        if (current_node_id not in net.outlinks):
            continue
        for i in range(len(net.outlinks[current_node_id])):
            cur_outlink = net.outlinks[current_node_id][i]
            new_cost = current_node_cost + cur_outlink.FFTT
            next_node_id = cur_outlink.en
            if (node_cost[next_node_id] > new_cost):
                node_cost[next_node_id] = new_cost
                heapq.heappush(heap,(new_cost,next_node_id))
                previous_node[next_node_id]=current_node_id
                previous_links_id[next_node_id] = cur_outlink.id
    
    return 0

"""Dial's algorithm"""
def dial(network,orig, dest, OD_vol):
    link_num = len(network.links)
    node_num = len(network.nodes)
    
    """Step1 Calculate node costs"""
    Dijkstra(network, orig, dest)
    
    """Step2 Calculate link likelihood"""
    for i in range(link_num):
        st = network.links[i].st
        en = network.links[i].en
        if ((network.nodes[en].cost - network.nodes[st].cost) > 0):
            network.links[i].likelihood = \
            exp(network.nodes[en].cost - network.nodes[st].cost - network.links[i].FFTT)
        else:
            network.links[i].likelihood = 0
    
    """Step3 Forward algorithm"""
    heap_NodeCost = []
    descend_NodeCost = []
    for i in range(node_num):        
        heapq.heappush(heap_NodeCost,(network.nodes[i].cost,i))
        
    while(len(heap_NodeCost) !=0):
        node_cost, node_id = heapq.heappop(heap_NodeCost)
        temp=(node_cost, node_id)
        descend_NodeCost.insert(0,temp)
        if (node_id not in network.outlinks):
            break
        for i in range(len(network.outlinks[node_id])):
            if (node_id == orig):
                network.outlinks[node_id][i].weight = \
                copy.deepcopy(network.outlinks[node_id][i].likelihood)
            else:
                sum_weight = 0
                for j in range(len(network.inlinks[node_id])):
                    sum_weight += network.inlinks[node_id][j].weight
                network.outlinks[node_id][i].weight = \
                (network.outlinks[node_id][i].likelihood)*sum_weight

    """Step4 Backward algorithm"""
    q = 1000
    for i in range(node_num):
        node_id = descend_NodeCost[i][1]
        
        sum_weight = 0
        if (node_id in network.inlinks):
            for j in range(len(network.inlinks[node_id])):
                sum_weight += network.inlinks[node_id][j].weight
        
        sum_travol = 0
        if (node_id in network.outlinks):
            for k in range(len(network.outlinks[node_id])):
                sum_travol += network.outlinks[node_id][k].tra_vol
        
        if (node_id not in network.inlinks):
            continue
        else:
            for l in range(len(network.inlinks[node_id])):
                network.inlinks[node_id][l].tra_vol = \
                (q + sum_travol)* (network.inlinks[node_id][l].weight) / sum_weight
        q = 0
        
    for i in range(link_num):
        print("linkID", network.links[i].id, ":", network.links[i].tra_vol)                  
    return None

In [82]:
"""Imput network data """

links = read_csv("dial_links2.csv")
nodes = read_csv("dial_nodes2.csv")
OD =  read_csv("dial_OD.csv")

"""Instantiate an object from Network class"""
network = Network()
for i in range(len(nodes)):
    temp_node = Node(nodes["nodeID"][i],nodes["X"][i],nodes["Y"][i])
    network.addNode(temp_node)

for j in range(len(links)):
    temp_link = Link(links["LInkID"][j],links["st"][j],links["en"][j],links["FFTT"][j])
    network.addLink(temp_link)

"""SUE using Dial's algorithm"""
dial(network,0, 8, 1000)

linkID 0 : 250.8011057677487
linkID 1 : 749.1988942322512
linkID 2 : 0.0
linkID 3 : 250.8011057677487
linkID 4 : 681.7480883659064
linkID 5 : 67.45080586634482
linkID 6 : 0.0
linkID 7 : 681.7480883659064
linkID 8 : 250.8011057677487
linkID 9 : 67.45080586634482
linkID 10 : 681.7480883659064
linkID 11 : 318.2519116340935
